In [ ]:
#| default_exp create_umap_layout

In [ ]:
#| export

import numpy as np
import umap
import daft

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Create umap layout(s)

In [ ]:
#| export

def umap_2d(embeddings: np.ndarray, n_neighbors: int = 15, min_dist: float = 0.1,
                metric: str = "cosine", random_state: int = 42) -> np.ndarray:
    """
    Create a UMAP layout from embeddings.

    Args:
        embeddings: np.ndarray, shape (n_samples, n_features)
        n_neighbors: int, default=15
        min_dist: float, default=0.1
        n_components: int, default=2
    """
    return umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist,
                     metric=metric, random_state=random_state).fit_transform(embeddings)

In [ ]:
#| hide

umap_2d(np.random.rand(5, 10), n_neighbors=15, min_dist=0.1, metric="cosine", random_state=42)

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


array([[-1.093321 , -4.5273743],
       [-2.5225453, -4.822378 ],
       [-1.7975024, -4.022668 ],
       [-1.2994211, -5.5270386],
       [-2.1556873, -5.7061405]], dtype=float32)

In [ ]:
#| export

def create_umap_col(df: daft.DataFrame, n_neighbors: int = 15, min_dist: float = 0.1,
                metric: str = "cosine", random_state: int = 42,
                col_namespace="umap") -> daft.DataFrame:
    """
    Create a UMAP layout from embeddings.

    Args:
        embeddings: np.ndarray, shape (n_samples, n_features)
        n_neighbors: int, default=15
        min_dist: float, default=0.1
        n_components: int, default=2
    """

    ## TODO: use namespacing for XYs column
    # embeds = np.asarray(df.select("embeddings").to_pylist()["embeddings"])
    embeds = df.select("embeddings").to_arrow()["embeddings"]
    shape = len(embeds), embeds.type.list_size
    embeds = embeds.values.to_numpy().reshape(shape)
    umap_xys = umap_2d(embeds, n_neighbors=n_neighbors, min_dist=min_dist,
                       metric=metric, random_state=random_state)
    umap_xys = daft.Series.from_numpy(umap_xys)
    return df.with_column(f"{col_namespace}_xys", umap_xys)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()